<a href="https://colab.research.google.com/github/Dare-Badejo-001/BioSteam-TEA-LCA/blob/main/TEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install biosteam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.5/429.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.4 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.6/936.6 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 54.6 MB/s eta

In [6]:
!pip install biorefineries

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for biorefineries: filename=biorefineries-2.29.0-py3-none-any.whl size=3778583 sha256=6a7dff2f7bab02b1a24d04f3b82513d888eb3034aea82237fa4a10916aa635a1
  Stored in directory: /root/.cache/pip/wheels/62/08/10/21ed1265fa45f714d9919158f56add186913158dd02fb771dd
Successfully built biorefineries


In [1]:
from warnings import filterwarnings; filterwarnings('ignore')
import biosteam as bst
import numpy as np
bst.nbtutorial()

ModuleNotFoundError: No module named 'biosteam'

## Inheriting from TEA objects


In [5]:
class SugarcaneTEA(bst.TEA):
    """
    Create a SugarcaneTEA object for techno-economic analysis of a biorefinery [1]_.

    Parameters
    ----------
    system : System
        Should contain feed and product streams.
    IRR : float
        Internal rate of return (fraction).
    duration : tuple[int, int]
        Start and end year of venture (e.g. (2018, 2038)).
    depreciation : str
        'MACRS' + number of years (e.g. 'MACRS7').
    operating_days : float
        Number of operating days per year.
    income_tax : float
        Combined federal and state income tax rate (fraction).
    lang_factor : float
        Lang factor for getting fixed capital investment from
        total purchase cost. If no lang factor, estimate capital investment
        using bare module factors.
    startup_schedule : tuple[float]
        Startup investment fractions per year
        (e.g. (0.5, 0.5) for 50% capital investment in the first year and 50%
        investment in the second).
    WC_over_FCI : float
        Working capital as a fraction of fixed capital investment.
    labor_cost : float
        Total labor cost (USD/yr).
    fringe_benefits : float
        Cost of fringe benefits as a fraction of labor cost.
    property_tax : float
        Fee as a fraction of fixed capital investment.
    property_insurance : float
        Fee as a fraction of fixed capital investment.
    supplies : float
        Yearly fee as a fraction of labor cost.
    maintenance : float
        Yearly fee as a fraction of fixed capital investment.
    administration : float
        Yearly fee as a fraction of fixed capital investment.

    References
    ----------
    .. [1] Huang, H., Long, S., & Singh, V. (2016). Techno-economic analysis of biodiesel
        and ethanol co-production from lipid-producing sugarcane. Biofuels, Bioproducts
        and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640

    """

    def __init__(self, system, IRR, duration, depreciation, income_tax,
                 operating_days, lang_factor, construction_schedule, WC_over_FCI,
                 labor_cost, fringe_benefits, property_tax,
                 property_insurance, supplies, maintenance, administration):
        # Huang et. al. does not take into account financing or startup
        # so these parameters are 0 by default
        super().__init__(system, IRR, duration, depreciation, income_tax,
                         operating_days, lang_factor, construction_schedule,
                         startup_months=0, startup_FOCfrac=0, startup_VOCfrac=0,
                         startup_salesfrac=0, finance_interest=0, finance_years=0,
                         finance_fraction=0, WC_over_FCI=WC_over_FCI)
        self.labor_cost = labor_cost
        self.fringe_benefits = fringe_benefits
        self.property_tax = property_tax
        self.property_insurance = property_insurance
        self.supplies= supplies
        self.maintenance = maintenance
        self.administration = administration

    # The abstract _DPI method should take installed equipment cost
    # and return the direct permanent investment. Huang et. al. assume
    # these values are equal
    def _DPI(self, installed_equipment_cost):
        return installed_equipment_cost

    # The abstract _TDC method should take direct permanent investment
    # and return the total depreciable capital. Huang et. al. assume
    # these values are equal
    def _TDC(self, DPI):
        return DPI

    # The abstract _FCI method should take total depreciable capital
    # and return the fixed capital investment. Again, Huang et. al.
    # assume these values are equal.
    def _FCI(self, TDC):
        return TDC

    # The abstract _FOC method should take fixed capital investment
    # and return the fixed operating cost.
    def _FOC(self, FCI):
        return (FCI*(self.property_tax + self.property_insurance
                     + self.maintenance + self.administration)
                + self.labor_cost*(1+self.fringe_benefits+self.supplies))

## Cash flow analysis and results

In [7]:
from biorefineries import sugarcane as sc

tea = SugarcaneTEA(system=sc.sugarcane_sys,
                 IRR=0.15,
                 duration=(2018, 2038),
                 depreciation='MACRS7',
                 income_tax=0.21, # Previously 35% in published study
                 operating_days=200,
                 lang_factor=3,
                 construction_schedule=(0.4, 0.6),
                 WC_over_FCI=0.05,
                 labor_cost=2.5e6,
                 fringe_benefits=0.4,
                 property_tax=0.001,
                 property_insurance=0.005,
                 supplies=0.20,
                 maintenance=0.01,
                 administration=0.005)

tea.show() # Print TEA summary at current options
# Ignore the warnings, these are taken care of internally.

SugarcaneTEA: sugarcane_sys
NPV: 1,000,073 USD at 15.0% IRR


In [8]:
tea.get_cashflow_table()

,Depreciable capital [MM$],Fixed capital investment [MM$],Working capital [MM$],Depreciation [MM$],Loan [MM$],...,Net earnings [MM$],Cash flow [MM$],Discount factor,Net present value (NPV) [MM$],Cumulative NPV [MM$]
2016,80.1,80.1,0,0,0,...,0,-80.1,1.15,-92.1,-92.1
2017,120,120,10,0,0,...,0,-130,1,-130,-222
2018,0,0,0,28.6,0,...,9.18,37.8,0.87,32.9,-189
2019,0,0,0,49,0,...,-8.81,40.2,0.756,30.4,-159
2020,0,0,0,35,0,...,5.21,40.2,0.658,26.5,-133
2021,0,0,0,25,0,...,12.8,37.8,0.572,21.6,-111
2022,0,0,0,17.9,0,...,17.7,35.5,0.497,17.7,-93.2
2023,0,3.19,0,17.9,0,...,17.7,32.3,0.432,14,-79.3
2024,0,0,0,17.9,0,...,17.7,35.5,0.376,13.4,-65.9
2025,0,0,0,8.93,0,...,24.7,33.7,0.327,11,-54.9


In [9]:
products = [bst.main_flowsheet('ethanol')]
costs = tea.production_costs(products)# USD/yr
np.round(costs / 1e6) # million USD / yr

array([58.])

In [10]:
feed = bst.main_flowsheet('sugarcane')
price = tea.solve_price(feed) # USD/kg
round(price, 5)

0.03467

In [11]:
tea.IRR = tea.solve_IRR()
tea.show()

SugarcaneTEA: sugarcane_sys
NPV: 4 USD at 15.1% IRR
